In [1]:
import pandas as pd
import random

In [2]:
from aizynthfinder.chem import Molecule, Reaction, MoleculeException
from aizynthfinder.training.utils import (
    Config,
    create_reactants_molecules,
    reverse_template,
    reaction_hash,
    reactants_to_fingerprint,
)
from aizynthfinder.utils.models import CUSTOM_OBJECTS, load_keras_model


In [3]:
data_reactions = pd.read_json("C:/Users/Yassine/Desktop/aizynth/uspto-reactions.json")
data_templates = pd.read_json("C:/Users/Yassine/Desktop/aizynth/uspto-templates.json")

In [4]:
data_templates_limited = data_templates.iloc[:100]
data_reactions_limited = data_reactions.iloc[:100]

In [5]:
data_templates_limited

,reaction_id,reactants,intra_only,products,reaction_smarts,dimer_only,necessary_reagent
0,0,Cl-[S;H0;D4;+0:1](-[C:2])(=[O;D1;H0:3])=[O;D1;...,0.0,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C:2])(=[...,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C:2])(=[...,0.0,
1,1,Cl-[S;H0;D4;+0:1](-[C;D1;H3:2])(=[O;D1;H0:3])=...,0.0,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C;D1;H3:...,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C;D1;H3:...,0.0,
2,2,Cl-[S;H0;D4;+0:1](-[C:2])(=[O;D1;H0:3])=[O;D1;...,0.0,[C:2]-[S;H0;D4;+0:1](=[O;D1;H0:3])(=[O;D1;H0:4...,[C:2]-[S;H0;D4;+0:1](=[O;D1;H0:3])(=[O;D1;H0:4...,0.0,
3,3,Cl-[S;H0;D4;+0:1](-[C:2])(=[O;D1;H0:3])=[O;D1;...,0.0,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C:2])(=[...,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C:2])(=[...,0.0,
4,4,Cl-[S;H0;D4;+0:1](-[C;D1;H3:2])(=[O;D1;H0:3])=...,0.0,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C;D1;H3:...,[C:5]-[O;H0;D2;+0:6]-[S;H0;D4;+0:1](-[C;D1;H3:...,0.0,
...,...,...,...,...,...,...,...
95,95,O-[CH3;D1;+0:1].[O;D1;H0:2]=[C:3]-[OH;D1;+0:4],0.0,[CH3;D1;+0:1]-[O;H0;D2;+0:4]-[C:3]=[O;D1;H0:2],[CH3;D1;+0:1]-[O;H0;D2;+0:4]-[C:3]=[O;D1;H0:2]...,0.0,
96,96,C-[c;H0;D3;+0:1](:[#7;a;+:2]):[c:3]:[#7;a;+:4],1.0,[#7;a;+:2]:[cH;D2;+0:1]:[c:3]:[#7;a;+:4],[#7;a;+:2]:[cH;D2;+0:1]:[c:3]:[#7;a;+:4]>>C-[c...,0.0,
97,97,C-[CH2;D2;+0:1]-[C:2],1.0,[C:2]-[CH3;D1;+0:1],[C:2]-[CH3;D1;+0:1]>>C-[CH2;D2;+0:1]-[C:2],0.0,
98,98,Cl-c1:c:c:c:c(-C(=[O;H0;D1;+0:1])-O-O):c:1.[C:...,0.0,[C:2]-[S;H0;D3;+0:3](-[C;D1;H3:4])=[O;H0;D1;+0:1],[C:2]-[S;H0;D3;+0:3](-[C;D1;H3:4])=[O;H0;D1;+0...,0.0,


In [6]:
data_reactions_limited

,_id,reactants,products,spectators,source,source_id
0,0,[Br:1][CH2:2][CH2:3][OH:4].[CH2:5]([S:7](Cl)(=...,[CH2:5]([S:7]([O:4][CH2:3][CH2:2][Br:1])(=[O:9...,C(N(CC)CC)C,uspto,d55b478b903cc66713bcd03da0cb590206c063770b2566...
1,1,[Br:1][CH2:2][CH2:3][CH2:4][OH:5].[CH3:6][S:7]...,[CH3:6][S:7]([O:5][CH2:4][CH2:3][CH2:2][Br:1])...,C(N(CC)CC)C,uspto,4f4596769c26752550f51dfa2e2ed3c85314e993098a21...
2,2,[CH2:1]([Cl:4])[CH2:2][OH:3].CCOCC.[CH2:10]([S...,[CH2:10]([S:14]([O:3][CH2:2][CH2:1][Cl:4])(=[O...,C(N(CC)CC)C,uspto,b437d8086077a93b6c04154fe4ea6b3b8a718cf617fcfd...
3,3,[Br:1][CH2:2][CH2:3][OH:4].[CH2:5]([S:7](Cl)(=...,[CH2:5]([S:7]([O:4][CH2:3][CH2:2][Br:1])(=[O:9...,C(N(CC)CC)C,uspto,11b36b516a4e353d293245bc8a3af922dc42c0e5d1e06c...
4,4,[Br:1][CH2:2][CH2:3][CH2:4][OH:5].[CH3:6][S:7]...,[CH3:6][S:7]([O:5][CH2:4][CH2:3][CH2:2][Br:1])...,C(N(CC)CC)C,uspto,ef05638cb9f9190bf222b9a92a7159e96ee090c12cf110...
...,...,...,...,...,...,...
95,95,[CH3:1][C:2]1[CH:21]=[CH:20][C:5]2[O:6][CH:7](...,[CH3:1][C:2]1[CH:21]=[CH:20][C:5]2[O:6][CH:7](...,,uspto,a888f8349b8e2090ad849fc83992ec014cbe8dd866835b...
96,96,CC1C(CSC)=[N+]([O-])C2C(=CC=CC=2)[N+]=1[O-].C[...,[CH3:31][S:29]([CH2:28][C:27]1[CH:18]=[N+:19](...,,uspto,e5eb326910ae3586b98dbb225489c01fba9135496e7c96...
97,97,C(SCC1C=[N+]([O-])C2C(=CC=CC=2)[N+]=1[O-])CC.[...,[CH2:18]([S:21]([CH2:23][C:24]1[CH:33]=[N+:32]...,,uspto,284a57e657a8469d48edcf166b75420be452de8b33826a...
98,98,ClC1C=CC=C(C(OO)=[O:9])C=1.[CH3:12][S:13][CH2:...,[CH3:12][S:13]([CH2:14][C:15]1[C:24]([C:25]([O...,C(Cl)(Cl)Cl,uspto,eea42804efc3e323c2b3c6ef130469926fcc00b667af56...


In [7]:
def get_random_row(lib):
    r = random.randint(0,lib.shape[0]-1)
    return [lib.iloc[r]]

In [9]:
def random_sample_lib(library, lib2):
    to_return = []
    for val, row in lib2.iterrows():
        if (type(row.reactants) == str):
            mols = create_reactants_molecules(row.reactants)
        else:
            continue
        try:
            ref_mol = Molecule(smiles=row.products, sanitize=True)
        except MoleculeException:
            continue
            
        new_product = None
        for template_row in get_random_row(library): #SAMPLER 
            #if row.template_hash == template_row.template_hash: #Check
            #    continue
            #smarts_fwd = reverse_template(template_row.retro_template) #Check
            if (type(template_row["reaction_smarts"]) != str):
                continue
            smarts_fwd = reverse_template(template_row["reaction_smarts"])
            try:
                new_product = Reaction(mols=mols, smarts=smarts_fwd).apply()[0][0]
            except (ValueError, IndexError):
                continue
            if new_product.basic_compare(ref_mol):
                continue
            break  # If we have reached here, we have found a match that fits all criteria

        if not new_product:
            continue
        
        to_return += [val,new_product]
    return to_return

In [10]:
def _sample_lib(library, lib2):
    to_return = []
    for val, row in lib2.iterrows():
        if (type(row.reactants) == str):
            mols = create_reactants_molecules(row.reactants)
        else:
            continue
        try:
            ref_mol = Molecule(smiles=row.products, sanitize=True)
        except MoleculeException:
            continue
            
        new_product = None
        for idx,template_row in library.iterrows(): #SAMPLER 
            #if row.template_hash == template_row.template_hash: #Check
            #    continue
            #smarts_fwd = reverse_template(template_row.retro_template) #Check
            if (idx != 15):
                continue
            if (type(template_row["reaction_smarts"]) != str):
                continue
            smarts_fwd = reverse_template(template_row["reaction_smarts"])
            try:
                new_product = Reaction(mols=mols, smarts=smarts_fwd).apply()[0][0]
            except (ValueError, IndexError):
                continue
            if new_product.basic_compare(ref_mol):
                continue
            break  # If we have reached here, we have found a match that fits all criteria

        if not new_product:
            continue
        
        to_return += [val,new_product]
    return to_return

In [37]:
y = random_sample_lib(data_templates, data_reactions)

In [38]:
z = y[1::2]

In [39]:
n = len(z)
arr_to_dat = np.empty(n, dtype="object")
for x in range(n):
    arr_to_dat[x] = data_reactions.iloc[y[2*x]].reactants + ">>" + z[x].smiles
arr_to_dat

array(['[F:1][C:2]1[CH:14]=[CH:13][C:5]([C:6]([CH2:8][CH2:9][C:10]([OH:12])=[O:11])=[O:7])=[CH:4][CH:3]=1.[N+:15]([O-])([OH:17])=[O:16]>>OCC[CH2:8][C:6]([c:5]1[cH:4][cH:3][c:2]([F:1])[cH:14][cH:13]1)=[O:7]',
       '[CH2:1]([O:8][C:9]1[CH:10]=[C:11]([CH:19]=[CH:20][C:21]=1[N+:22]([O-])=O)[C:12]([CH2:14][CH2:15][C:16]([OH:18])=[O:17])=[O:13])[C:2]1[CH:7]=[CH:6][CH:5]=[CH:4][CH:3]=1>>Nc1[c:9]([O:8][CH2:1][c:2]2[cH:3][cH:4][cH:5][cH:6][cH:7]2)[cH:10][c:11]([C:12](=[O:13])[CH2:14][CH2:15][C:16](=[O:17])[OH:18])[cH:19][cH:20]1',
       '[NH2:1][C:2]1[N:7]=[C:6]([NH2:8])[C:5]([CH2:9][C:10]2[CH:15]=[C:14]([O:16][CH2:17][CH3:18])[C:13]([C:19](=[O:25])[CH2:20][S:21]([CH3:24])(=[O:23])=[O:22])=[C:12]([O:26][CH2:27][CH3:28])[CH:11]=2)=[CH:4][N:3]=1.C(O)C.[BH4-].[Na+]>>Oc1[c:13]([C:19]([CH2:20][S:21](=[O:22])(=[O:23])[CH3:24])=[O:25])[c:12]([O:26][CH2:27][CH3:28])[cH:11][c:10]([CH2:9][c:5]2[cH:4][n:3][c:2]([NH2:1])[n:7][c:6]2[NH2:8])[cH:15]1',
       ...,
       '[N+:1]([C:4]1[CH:5]=[N:6][C:7]2[N: